In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

In [191]:
x1 = torch.tensor([[1],[2.]])
W1 = torch.tensor([[1,2.],
                   [-1,3]], requires_grad=True)
W2 = torch.tensor([[1,-1.],
                   [1,2]], requires_grad=True)
W3 = torch.tensor([[-1,2.],
                  [0,1]], requires_grad=True)
phi1 = torch.relu
phi2 = torch.sigmoid
phi3 = torch.softmax
l = lambda yh,t: - t.view(1,-1) @ torch.log(yh)
t = torch.tensor([[1], [0.]])
# same result:
# sl = nn.NLLLoss(reduction="sum")
# sl(torch.log_softmax(z3,0).view(1,-1), torch.max(t, 0)[1])
grad = lambda x: x.clone().detach().requires_grad_(True)

z1 = W1@x1
x2 = phi1(z1)
z2 = W2@x2
#z2 = grad(z2)
x3 = phi2(z2)
#x3 = grad(x3)
z3 = W3@x3
yh = phi3(z3, 0)
E = l(yh.flatten(),t) 
T = lambda A: A.transpose(0,1)

E.backward()
E

tensor([0.4741], grad_fn=<MvBackward>)

In [192]:
with torch.no_grad():
    dy = -t/yh
    dsoft = torch.tensor([[yh[0,0]*(1-yh[0,0]), -yh[0,0]*yh[1,0]],
                        [-yh[0,0]*yh[1,0], yh[1,0]*(1-yh[1,0])]])
    dsig = torch.diag((x3 * (1-x3)).flatten())
    drelu = torch.diag((z1 > 0).to(torch.float).flatten())
    
    dz3 = dsoft @ dy
    dz2 = dsig @ (T(W3) @ dz3)
    dz1 = drelu @ (T(W2) @ dz2)
    
    dw3 = dz3 @ T(x3)
    dw2 = dz2 @ T(x2)
    dw1 = dz1 @ T(x1)
    
    print(dw3)
    print(dw2)
    print(dw1)

tensor([[-0.1888, -0.3775],
        [ 0.1888,  0.3775]])
tensor([[ 4.7193e-01,  4.7193e-01],
        [-6.7510e-07, -6.7510e-07]])
tensor([[ 0.0944,  0.1888],
        [-0.0944, -0.1888]])


In [193]:
print(W3.grad)
print(W2.grad)
print(W1.grad)

tensor([[-0.1888, -0.3775],
        [ 0.1888,  0.3775]])
tensor([[ 4.7193e-01,  4.7193e-01],
        [-6.7510e-07, -6.7510e-07]])
tensor([[ 0.0944,  0.1888],
        [-0.0944, -0.1888]])


In [50]:
dz3

tensor([[-0.3775],
        [ 0.3775]])

In [52]:
dw2

tensor([[0., 0.],
        [0., 0.]])

In [47]:
W2.grad

tensor([[ 4.7193e-01,  4.7193e-01],
        [-6.7510e-07, -6.7510e-07]])

In [22]:
W3.grad

tensor([[-0.1888, -0.3775],
        [ 0.1888,  0.3775]])

In [23]:
dw3

tensor([[-0.1888, -0.3775],
        [ 0.1888,  0.3775]], grad_fn=<MmBackward>)